In [19]:
import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
import tensorflow as tf
import warnings
import xgboost as xgb
from lightgbm import LGBMRegressor
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.data import Dataset
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten, Conv1D, TimeDistributed, RepeatVector
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

In [20]:
# Função para criar dados multivariados
def multivariate_data(
    dataset: np.ndarray,
    target: np.ndarray,
    start_index: int,
    end_index: int,
    history_size: int,
    target_size: int,
    step: int,
    single_step: bool = False
) -> tuple[np.ndarray, np.ndarray]:
    """Create multivariate data for time series forecasting.

    This function generates input data and corresponding target data for time series forecasting tasks.
    It can be used for both single-step and multi-step forecasting.
    The function is based on the TensorFlow tutorial: https://www.tensorflow.org/tutorials/structured_data/time_series#part_2_forecast_a_multivariate_time_series 

    Args:
        dataset (np.ndarray): The input dataset, typically a 2D array with shape (num_samples, num_features).
        target (np.ndarray): The target variable or values to predict, typically a 1D array with shape (num_samples,).
        start_index (int): The starting index of the data in the dataset array.
        end_index (int): The ending index of the data in the dataset array.
        history_size (int): The number of time steps to consider as history for each input data point.
        target_size (int): The number of future time steps to predict.
        step (int): The number of time steps to skip between data points in the history.
        single_step (bool, optional): If True, create data for single-step forecasting; if False, create data for multi-step forecasting. Defaults to False.

    Returns:
        tuple[np.ndarray, np.ndarray]: A tuple containing:
            - A 3D array of shape (num_samples, history_size, num_features) containing input data.
            - A 1D or 2D array (depending on single_step) containing target data.
              If single_step is True, it's a 1D array of shape (num_samples,) with the next time step's target.
              If single_step is False, it's a 2D array of shape (num_samples, target_size) with multi-step targets.

    Example:
        >>> data, labels = multivariate_data(
        ...     dataset=data_array,
        ...     target=target_array,
        ...     start_index=0,
        ...     end_index=100,
        ...     history_size=10,
        ...     target_size=1,
        ...     step=1,
        ...     single_step=True
        ... )
    """
    data = []
    labels = []
    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i : i + target_size])

    return np.array(data), np.array(labels)

def metric_display(y_test, y_pred, idx_start = None, idx_end = None) -> None :
    """Exibe métricas de avaliação de um modelo de regressão e gera um gráfico de comparação.

    Esta função calcula várias métricas de avaliação de um modelo de regressão, incluindo RMSE, MAE, MAPE, SMAPE, R² e EVS.
    Além disso, ela cria um gráfico de linha para visualizar a comparação entre os valores reais (y_test) e os valores previstos (y_pred).

    Args:
        y_test (array-like): Os valores reais da variável de destino.
        y_pred (array-like): Os valores previstos pelo modelo.
        idx_start (str): A data e hora de início do período de teste.
        idx_end (str): A data e hora de término do período de teste.

    Returns:
        None

    Examples:
        >>> y_test = [3.0, 4.0, 5.0, 6.0, 7.0]
        >>> y_pred = [2.8, 4.2, 4.9, 6.3, 7.2]
        >>> metric_display(y_test, y_pred, idx_start='2021-01-01 00:00', idx_end='2021-01-01 04:00')
        
    """
    # Calcula o erro médio quadrático (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Calcula o erro absoluto médio (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Calcula o erro percentual absoluto médio (MAPE)
    mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100
    
    # Calcula o erro percentual absoluto médio simétrico (SMAPE)
    smape = 2 * np.mean(np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test))) * 100
    
    # Calcula o coeficiente de determinação (R²)
    r_squared = r2_score(y_test, y_pred)
    
    # Calcula a pontuação de variância explicada (EVS)
    evs = explained_variance_score(y_test, y_pred)

    print(f'[Menor é melhor] RMSE: {rmse:.4f}')
    print(f'[Menor é melhor] MAE: {mae:.4f}')
    print(f'[Menor é melhor] MAPE: {mape:.4f}%')
    print(f'[Menor é melhor] SMAPE: {smape:.4f}% ')
    print(f'[Maior é melhor] R²: {r_squared:.4f} ')
    print(f'[Maior é melhor] EVS: {evs:.4f} ')

    # Cria um índice para o período de teste com base nos parâmetros fornecidos
    if idx_start is not None and idx_end is not None:
        # Se as datas forem fornecidas, use-as para criar o índice de hora em hora
        idx = pd.date_range(start=idx_start, end=idx_end, freq='H')
    else:
        # Caso contrário, use índices numéricos padrão com base no comprimento dos dados
        idx = range(len(y_test))

    # Cria um DataFrame com o índice calculado
    df = pd.DataFrame({'Real (y_test)': y_test.ravel(), 'Previsto (y_pred)': y_pred.ravel()}, index=idx)

    # Gera um gráfico de comparação
    fig = px.line(df, title='Comparação entre y_test e y_pred',
                   labels={'value': 'Valor', 'variable': 'Valor'},
                   line_shape='linear')
    if idx_start is not None and idx_end is not None:
        fig.update_xaxes(title_text='Data e Hora', tickformat="%Y-%m-%d %H:%M")

    # Exibe o gráfico
    fig.show()

def plot_training_history(history):
    """
    Plota a história de treinamento de um modelo Keras usando Plotly. 

    Args:
        history (keras.callbacks.History): O objeto de história de treinamento do modelo Keras.

    Returns:
        None
    """
    # Extrair informações da história de treinamento
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Converter o objeto de intervalo em lista
    epochs = list(range(1, len(train_loss) + 1))

    # Criar um gráfico interativo com proporção quadrada
    fig = go.Figure()

    # Adicionar curva de perda de treinamento
    fig.add_trace(go.Scatter(x=epochs, y=train_loss, mode='lines', name='Train Loss'))

    # Adicionar curva de perda de validação
    fig.add_trace(go.Scatter(x=epochs, y=val_loss, mode='lines', name='Validation Loss'))

    # Configurar o layout do gráfico
    fig.update_layout(
        title='Loss Over Epochs',
        xaxis_title='Epochs',
        yaxis_title='Loss',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        width=1000,
        height=600
    )

    # Exibir o gráfico
    fig.show()

# Carregar os dados
data = pd.read_csv('../TCC/datasets/demanda.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp']).dt.strftime('%Y-%m-%d %H:%M')
data.set_index('Timestamp', inplace=True)

# Definindo a coluna alvo
target = 'kVA fornecido'

# Copiando o dataframe original para preservar os dados originais
df_final = data.copy()

# Definir o caminho para salvar os modelos
model_path = f'../TCC/modelos/{target.replace(" ", "_")}_'

# Criação dos conjuntos de treinamento, validação e teste
train_end_idx = df_final.index.get_loc('2023-01-01 00:00')  # Índice de término para treinamento
valid_end_idx = df_final.index.get_loc('2023-02-15 00:00')  # Índice de término para validação
test_end_idx = df_final.index.get_loc('2023-02-21 23:00')   # Índice de término para teste

# Separação dos dados de entrada e saída
X = df_final[df_final.columns.drop(target)].values # Dados de entrada
y = df_final[target].values.reshape(-1, 1)  # Dados de saída

# Inicialização e ajuste dos scalers para normalização dos dados
scaler_X = MinMaxScaler(feature_range=(0, 1))   # Inicialização do scaler para normalização dos dados de entrada
scaler_y = MinMaxScaler(feature_range=(0, 1))   # Inicialização do scaler para normalização dos dados de saída
scaler_X.fit(X[:train_end_idx]) # Ajuste ao conjunto de treinamento
scaler_y.fit(y[:train_end_idx]) # Ajuste ao conjunto de treinamento
X_norm = scaler_X.transform(X)  # Normalização dos dados de entrada
y_norm = scaler_y.transform(y)  # Normalização dos dados de saída

# Redução de dimensionalidade usando PCA
pca = PCA().fit(X_norm[:train_end_idx]) # Ajuste aos dados normalizados de entrada
num_components = len(pca.explained_variance_ratio_) # Número de componentes principais

# Plotagem da variância explicada pelo número de componentes principais
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, num_components+1), y=np.cumsum(pca.explained_variance_ratio_), mode='lines', name='Cumulativa', marker_color='gray'))
fig.add_trace(go.Bar(x=np.arange(1, num_components+1), y=pca.explained_variance_ratio_, name='Individual', marker_color='blue'))
fig.update_layout(title='Variância explicada pelo número de componentes principais',
                  xaxis_title='Número de componentes principais',
                  yaxis_title='Variância explicada',
                  autosize=False, width=1000, height=600)
fig.show()

# Redução de dimensionalidade usando PCA com 80% de variância explicada mantida
pca = PCA(n_components=0.80).fit(X_norm[:train_end_idx]) # Ajuste aos dados normalizados de entrada
X_pca = pca.transform(X_norm) # Transformação dos dados normalizados de entrada

# Concatenação das características reduzidas pelo PCA e dos alvos normalizados
dataset_norm = np.concatenate((X_pca, y_norm), axis=1) # Concatenação das características reduzidas pelo PCA e dos alvos normalizados

# Definição do tamanho da janela de histórico e do tamanho do alvo futuro
past_history = 24 # Número de horas anteriores utilizado para prever o dado futuro
future_target = 0 # Se for 0, o modelo será de regressão, caso contrário, será de classificação

# Criação de conjuntos de treinamento, validação e teste usando a função 'multivariate_data'
X_train, y_train = multivariate_data(dataset_norm, dataset_norm[:, -1], 0, train_end_idx, past_history, future_target, step=1, single_step=True)
X_val, y_val = multivariate_data(dataset_norm, dataset_norm[:, -1], train_end_idx, valid_end_idx, past_history, future_target, step=1, single_step=True)
X_test, y_test = multivariate_data(dataset_norm, dataset_norm[:, -1], valid_end_idx, test_end_idx, past_history, future_target, step=1, single_step=True)

# Definição do tamanho do lote e do tamanho do buffer
batch_size = 32 # Número de amostras por atualização de gradiente (batch), menor mais lento, mas mais preciso (evita overfitting) 
buffer_size = 1000 # Tamanho do buffer para embaralhar os dados de treinamento, utilizado para datasets muito grandes

# Criação de conjuntos de dados TensorFlow para treinamento e validação
# tf.data.Dataset: representa um conjunto de elementos, no qual cada elemento possui o mesmo tipo
# from_tensor_slices: cria um Dataset a partir de um tensor
# cache: mantém os dados na memória para acelerar o treinamento
# shuffle: embaralha os dados para o treinamento
# batch: cria lotes de dados para o treinamento
# prefetch: prepara os dados para o treinamento
train = Dataset.from_tensor_slices((X_train, y_train)).cache().shuffle(buffer_size).batch(batch_size).prefetch(1) # Conjunto de treinamento
validation = Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(1) # Conjunto de validação

# Definição de parâmetros comuns aos modelos
input_shape = X_train.shape[-2:] # Formato dos dados de entrada (número de amostras, número de características)
loss = MeanSquaredError() # Função de perda (erro quadrático médio)
metric = [RootMeanSquaredError()] # Métrica (raiz quadrada do erro quadrático médio)
early_stopping = EarlyStopping(patience=10) # Parada antecipada (sem melhoria no erro quadrático médio por 10 épocas)

# Reformulação de 'y_test' para sua forma original e inversão da normalização
y_test = y_test.reshape(-1, 1) # Reformulação de 'y_test' para sua forma original
y_test_inv = scaler_y.inverse_transform(y_test) # Inversão da normalização de 'y_test'

# # Limpeza do ambiente do TensorFlow
# clear_session()

# # Definição do modelo
# multivariate_lstm = Sequential([
#     LSTM(100, input_shape=input_shape, return_sequences=True),
#     Flatten(),
#     Dense(200, activation='relu'),
#     Dropout(0.1),
#     Dense(1)
# ])

# # Definição de Callbacks
# model_checkpoint = ModelCheckpoint(f'{model_path}multivariate_lstm.h5', monitor=('val_loss'), save_best_only=True)
# optimizer = Adam(learning_rate=6e-3, amsgrad=True)

# # Compilação do modelo
# multivariate_lstm.compile(loss=loss, optimizer=optimizer, metrics=metric)

# # Treinamento do modelo
# history = multivariate_lstm.fit(train, epochs=120, validation_data=validation, callbacks=[early_stopping, model_checkpoint])

# # Plotagem da história de treinamento
# plot_training_history(history)

# Carregamento do modelo com melhor desempenho
multivariate_lstm = load_model(f'{model_path}multivariate_lstm.h5')

# Geração de previsões usando o modelo treinado
forecast = multivariate_lstm.predict(X_test)
lstm_pred = scaler_y.inverse_transform(forecast)

# Exibição das métricas de avaliação e do gráfico de comparação
metric_display(y_test_inv, lstm_pred, df_final.index[valid_end_idx+past_history], df_final.index[test_end_idx-1])

5/5 [==============================] - 0s 5ms/step
[Menor é melhor] RMSE: 223.2664
[Menor é melhor] MAE: 158.3072
[Menor é melhor] MAPE: 11.7938%
[Menor é melhor] SMAPE: 10.9413% 
[Maior é melhor] R²: 0.7448 
[Maior é melhor] EVS: 0.7619 


In [68]:
# Preparação dos dados para prever os próximos dias
X_last = X_test[-1].reshape(1, past_history, X_test[-1].shape[1])  # Use os últimos valores disponíveis nos dados de teste
y_future = []  # Armazenará os valores previstos do target

n_dias = 1  # Número de dias para prever
n_steps = 24 * n_dias  # Número de passos à frente para prever

# Faz previsões para os próximos dias
for i in range(n_steps):  # Supondo que n_steps é o número de passos à frente que você deseja prever
    prediction = multivariate_lstm.predict(X_last)  # Faça a previsão para o próximo dia
    X_last = np.roll(X_last, shift=-1, axis=1)  # Desloca os dados em X_last para incluir a nova previsão
    X_last[0, -1, :] = prediction  # Insere a previsão nas últimas posições de X_last
    y_future.append(prediction)

# Desfaz a normalização para obter os valores reais previstos do target
y_future = scaler_y.inverse_transform(np.array(y_future).reshape(-1, 1))

1/1 [==============================] - 0s 22ms/step


In [69]:
date_start = '2023-02-16 00:00'
date_end = '2023-02-21 22:00'
date = pd.date_range(start=date_start, end=date_end, freq='H')

# Cria um DataFrame com o índice calculado
df_test = pd.DataFrame({'Previsto (lstm_pred)': lstm_pred.ravel()}, index=date)

# Calcule a nova data final com base no tamanho de y_future
new_date_end = pd.to_datetime(date_end) + pd.Timedelta(hours=len(y_future))

# Cria um novo índice com base na data final calculada
new_date = pd.date_range(start=date_start, end=new_date_end, freq='H')

# Aloca os valores futuros previstos em um DataFrame com o novo índice a partir de 'date_end' + 1 hora
start_date = new_date.get_loc(date_end) + 1
df_future = pd.DataFrame({'Futuro (lstm_pred)': y_future.ravel()}, index=new_date[start_date:])

# Converter new_date_end, que é uma timestamp, para uma string
data_string = new_date_end.strftime("%Y-%m-%d %H:%M")

# Obtem os valores reais
real = df_final[target][valid_end_idx+24:df_final.index.get_loc(data_string)+1]

# Concatena os DataFrames de teste e futuro
df_test = pd.concat([df_test, real, df_future])

# Renomear coluna, 0 para 'Real (y_test)'
df_test.rename(columns={0: 'Real (y_test)'}, inplace=True)

# Gera um gráfico de comparação
fig = px.line(df_test, title='Comparação entre y_test_inv e lstm_pred', labels={'value': 'Valor', 'variable': 'Valor'}, line_shape='linear')
fig.update_xaxes(title_text='Data e Hora', tickformat="%Y-%m-%d %H:%M")

# Exibe o gráfico
fig.show()